# VQEで水素分子の基底エネルギーを求める

$\newcommand{\ket}[1]{|#1\rangle}$
$\newcommand{\braket}[2]{\langle #1 | #2 \rangle}$

VQE (variational quantum eigensolver; 変分量子固有値ソルバー)を使って実際の量子化学の問題を解いてみましょう。

VQEは2014年に発表された手法（[Peruzzo et al.](https://doi.org/10.1038/ncomms5213)）で、超伝導型量子コンピュータを使って水素分子などの小さい分子の基底状態エネルギーを求める問題に応用されたのは2017年の[Kandala et al.](https://doi.org/10.1038/nature23879)が始めてです。2017年というと量子コンピュータ業界においては太古の出来事ですが、それでも実際の学術論文の題材になった問題を実機で試すのは面白いのではないかと思います。

このコースは量子化学の講義ではないので、今回は問題のセットアップには深入りしません。幸い、Qiskit Nature（Qiskitの自然科学計算関連のモジュール）にいろいろなツールが組み込まれており、扱いたい分子の情報だけから量子コンピュータで期待値を測定すべき演算子を導出するところまでは自動でできます。この実習では、

- VQEを行うための変分量子回路を書く
- 渡された演算子の期待値を計算し、量子回路が作る量子状態のエネルギー期待値を計算する関数を書く
- Qiskit Runtimeを使って、実機での計算結果をもとにエネルギーを最小化するように変分回路のパラメータを調整していく

という部分を扱います。

In [ ]:
import sys
import time
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, VQE
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit import ParameterVector
from qiskit.circuit.random import random_circuit
from qiskit.primitives import BackendEstimator, Estimator
from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit_aer import AerSimulator
from qiskit_ibm_provider import IBMProvider, least_busy
from qiskit_ibm_provider.accounts import AccountNotFoundError
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Estimator as RuntimeEstimator
from qiskit_nature import settings
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.second_q.mappers import JordanWignerMapper

settings.use_pauli_sum_op = False

sys.path.append('/home/jovyan/qc-workbook-lecturenotes')
from qc_workbook.utils import operational_backend
from qc_workbook.show_state import show_state

## （背景知識）水素分子を解くとは？

量子化学の背景とVQEとのつながりについての詳しい説明は、この実習の発想元になっている[昨年のIBM Quantum Challengeのexercise 4](https://github.com/qiskit-community/ibm-quantum-spring-challenge-2022/blob/main/exercise4/04.quantum_chemistry.ipynb)（[日本語版](https://github.com/qiskit-community/ibm-quantum-spring-challenge-2022/blob/main/exercise4/04.quantum_chemistry-ja.ipynb)）が参考になります。ここでは最低限の説明をします。

今やりたいことは、ある分子の基本情報（構成する原子核種と数、電子の数、原子核の位置）を与えられた時に、その分子のエネルギー基底状態のエネルギー固有値を求めることです。ここで、原子核は電子に比べて遥かに重いことを利用し、原子核の位置を手で与えるパラメータとしています（Born-Oppenheimer approximation）。この基底状態エネルギーが求められれば、例えば今度は原子核の位置を動かしながら同じ計算を繰り返すことによって、最も基底状態エネルギーの低くなる原子核の配置（＝自然界で実現される分子のエネルギー基底状態の良い近似）を探すことなどができます。

### 分子軌道と$n$電子系の波動関数

原子単体に対しては、電子の軌道（エネルギー固有波動関数）$\{\chi_j(\mathbf{r})\}_j$とそれらに対応するエネルギーを、古典コンピュータでも精度良く計算することができます。ここで$\mathbf{r}$は電子の位置を表し、$j$はエネルギー固有状態のインデックスです。さらに分子中の一電子軌道（Molecular orbital）$\phi_m(\mathbf{r})$（$m$は軌道のインデックス）も、分子を構成する原子における軌道の重ね合わせとして

$$
\phi_m(\mathbf{r}) = \sum_j D_{mj}\chi_j(\mathbf{r})
$$

のように表現できます。

理論的には原子軌道も分子軌道も無数に存在しますが、今の問題のような場合、固有エネルギーの高い軌道はほとんど影響を持たないので、有限個の$\phi_m$のみを扱います。また、電子にはスピンの自由度があり、内部に量子ビットのような二次元系を持ちますが、それも軌道の一部とみなしていいので（spin orbitals）、結局、固有エネルギーで順序づけして下から$2M$個の分子軌道を考えることになります。

一般に分子には複数の電子がありますが、パウリの排他律から、各軌道には最大1個の電子しか入りません。従って、$n$電子系の波動関数は$n$個の占有されている軌道の掛け算で表現されます。ただし、同じくパウリの排他律の帰結として、全系の波動関数は電子の交換について反対称になるので、単純な掛け算ではなくSlater行列式$| \phi_{m_1} \phi_{m_2} \dots \phi_{m_n} | (\mathbf{r}_1, \mathbf{r}_2, \dots, \mathbf{r}_n) $が用いられます。一般の$n$電子系の状態はこのSlater行列式を基底とする線形空間の元と対応づけられて、

$$
\psi_{\text{elec}}(\mathbf{r}_1, \mathbf{r}_2, \dots,\mathbf{r}_n) = \sum_{m_1, m_2,\dots,m_n} C_{m_1, m_2, \dots, m_n} | \phi_{m_1}\phi_{m_2} \dots \phi_{m_n} | (\mathbf{r}_1, \mathbf{r}_2, \dots, \mathbf{r}_n)
$$

と書けます。特に、$n$電子系のエネルギー固有状態も上のようなSlater行列式の重ね合わせになります（電子同士の相互作用によって、全体のエネルギー固有状態は一電子のエネルギー固有状態の積にはなりません）。したがって、エネルギー固有状態や固有値を求めることは上の係数$C_{m_1, m_2, \dots, m_n}$を求めることに帰着します。ところが、軌道$\phi_m$が$2M$個あり、電子が$n$個あれば、系は$\binom{2M}{n}$次元となり、古典コンピュータでの計算はすぐに破綻してしまいます。そこで量子コンピュータの出番となるわけです。

### 第二量子化表現とJordan-Wigner変換

分子軌道の問題を量子コンピュータで扱う場合、系の状態を$n$体の波動関数$\psi_{\text{elec}}(\mathbf{r}_1, \mathbf{r}_2, \dots,\mathbf{r}_n)$で表現するよりも、$2M$個の軌道の占有・非占有で表現（第二量子化）する方が自然なマッピングとなります。その場合、ハミルトニアンは分子軌道ごとに定義される「生成・消滅演算子」$\hat{a}_m, \hat{a}_m^{\dagger}$を使って書かれ、

$$
\hat{H} =\sum_{r s} h_{r s} \hat{a}_{r}^{\dagger} \hat{a}_{s}
+\frac{1}{2} \sum_{p q r s} g_{p q r s} \hat{a}_{p}^{\dagger} \hat{a}_{q}^{\dagger} \hat{a}_{r} \hat{a}_{s}+E_{N}
$$

となります。ここで$h_{rs}$、$g_{pqrs}$、$E_N$は全て分子の構造から決まる（古典的に計算できる）定数で、それぞれ一電子のエネルギー、二電子間の電気的な反発のエネルギー、原子核同士の反発エネルギーに対応します。

生成・消滅演算子は、軌道$m$が占有されていない状態を$\ket{u}_m$、されている状態を$\ket{o}_m$とすると、

$$
\begin{align}
\hat{a}_m \ket{u}_m & = 0, \\
\hat{a}_m \ket{o}_m & = \ket{u}_m, \\
\hat{a}_m^{\dagger} \ket{u}_m & = \ket{o}_m, \\
\hat{a}_m^{\dagger} \ket{o}_m & = 0
\end{align}
$$

のように作用します。

このような占有・非占有の2次元系は、課題3のSchwingerモデルの問題にもあった通り、そのまま量子ビットに対応づけられます。つまり、$2M$個の軌道の状態を$2M$ビットの量子レジスタで表現できます。Schwingerモデルの問題ではサイトに粒子が存在する状態を$\ket{0}$、存在しない状態を$\ket{1}$に対応づけしましたが、今回はQiskitの習わしに従って、軌道が占有されている状態を$\ket{1}$、占有されていない状態を$\ket{0}$で表します。すると、$\hat{a}$と$\hat{a}^{\dagger}$はそれぞれ$\frac{1}{2}(X + iY)$と$\frac{1}{2}(X - iY)$に対応します。

$$
\begin{align}
\frac{1}{2}(X + iY) \ket{0} & = \frac{1}{2} (\ket{1} - \ket{1}) = 0, \\
\frac{1}{2}(X + iY) \ket{1} & = \frac{1}{2} (\ket{0} + \ket{0}) = \ket{0}, \\
\frac{1}{2}(X - iY) \ket{0} & = \frac{1}{2} (\ket{1} + \ket{1}) = \ket{1}, \\
\frac{1}{2}(X - iY) \ket{1} & = \frac{1}{2} (\ket{0} - \ket{0}) = 0.
\end{align}
$$

ただし、異なる軌道$j$と$k$の生成・消滅演算子が「反交換」しなければいけない（$a_r a_s = -a_s a_r$）という物理的要請があるので、実際には軌道$j$の生成・消滅演算子は

$$
\begin{align}
a_j \rightarrow I_{2M-1} \otimes I_{2M-2} \otimes \dots \otimes \frac{X_j + iY_j}{2} \otimes Z_{j-1} \otimes \dots \otimes Z_{0}, \\
a_j^{\dagger} \rightarrow I_{2M-1} \otimes I_{2M-2} \otimes \dots \otimes \frac{X_j - iY_j}{2} \otimes Z_{j-1} \otimes \dots \otimes Z_{0}
\end{align}
$$

のように、$j$より小さい軌道のスロットに$Z$演算子を挿入したような量子レジスタの演算に対応します。このような対応づけをJordan-Wigner変換といいます。実はSchwingerモデルの問題でも同じ操作をしていますが、説明を割愛していました。

Qiskit Natureのさまざまなクラスを利用すると、分子の核種と三次元位置を入力するだけで、ハミルトニアンの定数部分を計算し、生成・消滅演算子をJordan-Wigner変換して、ハミルトニアンをパウリ積（$X, Y, Z$ゲートと恒等演算$I$の組み合わせ）の和にしてくれます。

In [ ]:
molecule = MoleculeInfo(
    ['H', 'H'],
    # 原子核の座標（単位Å）
    [(0.0, 0.0, -0.3695), (0.0, 0.0, 0.3695)],
    charge=0,
    multiplicity=1  # = 2*spin + 1
)

# sto3g: 原子軌道の計算手法
driver = PySCFDriver.from_molecule(molecule, basis='sto3g')

# 解くべき問題を生成
problem = driver.run()

# 内殻軌道を固定し、問題の次元を削減
problem = FreezeCoreTransformer().transform(problem)

# 問題を表すハミルトニアンを第二量子化形式で表示
hamiltonian = problem.hamiltonian.second_q_op()
print(hamiltonian)

上で`basis="sto3g"`で[STO-3G](https://en.wikipedia.org/wiki/STO-nG_basis_sets)という原子軌道の計算（展開）方法を指定しています。STO-3Gでは最低限必要な軌道しか計算に含まないので、$2M=2n$となり、水素分子の場合は$n=2$なので4分子軌道のハミルトニアンが導出されます。軌道の番号と実際の分子軌道との対応づけは

- $0$から$n-1$番軌道がスピン$\alpha$の軌道
- $n$から$2n-1$番軌道がスピン$\beta$の軌道

で、それぞれエネルギーの低い順に並んでいます。$\alpha$と$\beta$は、何か適当な方向を基底に決めた時のスピンの二つの固有状態を表します。

上のハミルトニアンのプリントアウトでは、例えば`+_0`は第0軌道の生成演算子、`-_3`は第3軌道の消滅演算子を表します。

次にこのハミルトニアンをJordan-Wigner変換し、これまで講義で扱ってきたSparsePauliOp（パウリ積の和）オブジェクトに落とし込みます。

In [ ]:
ham_op = JordanWignerMapper().map(hamiltonian)
print(ham_op)

## 量子計算部分

### 期待値の計算

量子コンピュータにおける観測量の期待値の計算方法についてこれまで何度か触れてきましたが、今回はより複雑な計算となるので、ステップを追って確認してみましょう。

#### おさらい：$X, Y, Z$の期待値

適当な1ビット量子回路で作られる量子状態での$X, Y, Z$の期待値を計算しましょう。

量子コンピュータで観測量の期待値を計算するには、まず観測量を演算子で表し、その固有ベクトルと固有値を求め、量子回路の終状態に対して固有ベクトルが計算基底に移るような基底変換をし、測定の結果得られる計算基底の確率分布と固有値を掛け合わせるのでした。

定義上、$Z$の固有ベクトルとは計算基底そのもので、固有値は$\pm 1$（$Z\ket{0} = \ket{0}, Z\ket{1} = -\ket{1}$）です。したがって、量子回路の終状態をそのまま測定し、0が出る確率$P_0$から1が出る確率$P_1$を引けば、$Z$の期待値が求まります。式で書けば

$$
\langle Z \rangle = P_0 - P_1
$$

です。

$X$の固有値$\pm 1$の固有ベクトルは$\frac{1}{\sqrt{2}} (\ket{0} \pm \ket{1})$なので、アダマールゲートで基底変換ができます。

$Y$の固有ベクトルは$\frac{1}{\sqrt{2}}(\ket{0} \pm i\ket{1})$なので、$S^{\dagger} = P(-\pi/2)$とアダマールゲートで基底変換ができます。$S^{\dagger}$ゲートのqiskitでの記法は`circuit.sdg(0)`です。

In [ ]:
simulator = AerSimulator()
shots = 10000

In [ ]:
# ランダムな1ビット回路
circuit = random_circuit(num_qubits=1, depth=3)

# 状態ベクトルから計算する厳密解
exact = {}
for op in ['X', 'Y', 'Z']:
    observable = SparsePauliOp(op)
    exact[op] = Statevector(circuit).expectation_value(observable).real

observed = {}

# circuitのコピーを3つ作り、それぞれに基底変換と測定を加え、simulatorで実行して、得られるカウントから期待値を計算してください。
##################
### EDIT BELOW ###
##################

circuits = list(circuit.copy() for _ in range(3))

circuits[0].h(0)
circuits[1].sdg(0)
circuits[1].h(0)
for circ in circuits:
    circ.measure_all()

result = simulator.run(transpile(circuits, backend=simulator), shots=shots).result()
for iop, op in enumerate(['X', 'Y', 'Z']):
    counts = result.get_counts(iop)
    observed[op] = (counts.get('0', 0) - counts.get('1', 0)) / shots

##################
### EDIT ABOVE ###
##################

print(f'QC: {observed}')
print(f'Exact: {exact}')

上のセルを何度か実行してみて、毎回`observed`と`exact`が統計誤差の範囲で大体一致することを確認してください。

#### $I$の期待値

恒等演算子を観測量としたとき、その期待値はいくらになるでしょうか？

#### $ZIZI$の期待値

今度は4ビットの回路で観測量$ZIZI$の期待値を計算します。固有値と固有ベクトルはどうなっているでしょうか？基底変換は必要でしょうか？

In [ ]:
# ランダムな4ビット回路
circuit = random_circuit(num_qubits=4, depth=5)

# 状態ベクトルから計算する厳密解
observable = SparsePauliOp('ZIZI')
exact = Statevector(circuit).expectation_value(observable).real

observed = None

# circuitを使って期待値observedを計算してください。
##################
### EDIT BELOW ###
##################

circuit.measure_all()

counts = simulator.run(transpile(circuit, backend=simulator), shots=shots).result().get_counts()
observed = 0.
for key, value in counts.items():
    sign = 1.
    if key[0] == '1':
        sign *= -1.
    if key[2] == '1':
        sign *= -1.
    observed += sign * value
observed /= shots

##################
### EDIT ABOVE ###
##################

print(f'QC: {observed}')
print(f'Exact: {exact}')

#### $ZIZI + 2IZII + 3XXYY$の期待値

今度の観測量はパウリ積の和です。量子回路ではこれを一度に計算することができませんが、和の期待値は期待値の和なので、それぞれの項について独立に期待値を計算し、最後に係数をかけて足し合わせれば同じ答えが得られます。量子回路はいくつ必要でしょうか？

In [ ]:
# ランダムな4ビット回路
circuit = random_circuit(num_qubits=4, depth=5)

# 状態ベクトルから計算する厳密解
observable = SparsePauliOp(['ZIZI', 'IZII', 'XXYY'], [1., 2., 3.])
exact = Statevector(circuit).expectation_value(observable).real

observed = None

# circuitを使って期待値observedを計算してください。
##################
### EDIT BELOW ###
##################

circuit_z = circuit.copy()
circuit_z.measure_all()

counts = simulator.run(transpile(circuit_z, backend=simulator), shots=shots).result().get_counts()
expval_zizi = 0.
expval_izii = 0.
for key, value in counts.items():
    sign = 1.
    if key[0] == '1':
        sign *= -1.
    if key[2] == '1':
        sign *= -1.
    expval_zizi += sign * value

    sign = 1.
    if key[1] == '1':
        sign *= -1.
    expval_izii += sign * value
expval_zizi /= shots
expval_izii /= shots

circuit_xy = circuit.copy()
circuit_xy.h([2, 3])
circuit_xy.sdg([0, 1])
circuit_xy.h([0, 1])
circuit_xy.measure_all()

counts = simulator.run(transpile(circuit_xy, backend=simulator), shots=shots).result().get_counts()
expval_xxyy = 0.
for key, value in counts.items():
    sign = 1.
    if key[0] == '1':
        sign *= -1.
    if key[1] == '1':
        sign *= -1.
    if key[2] == '1':
        sign *= -1.
    if key[3] == '1':
        sign *= -1.
    expval_xxyy += sign * value
expval_xxyy /= shots

observed = expval_zizi + 2. * expval_izii + 3. * expval_xxyy

##################
### EDIT ABOVE ###
##################

print(f'QC: {observed}')
print(f'Exact: {exact}')

#### Estimatorの利用と水素分子ハミルトニアンの期待値

上の`ham_op`の期待値を、適当な4ビット回路について計算しましょう。期待値の手計算の仕方が確認できたので、これ以降は`Estimator`を使います。

In [ ]:
# ランダムな4ビット回路
circuit = random_circuit(num_qubits=4, depth=5)

# 状態ベクトルから計算する厳密解
exact = Statevector(circuit).expectation_value(ham_op).real

# 回路からEstimatorを使って計算する期待値
estimator = BackendEstimator(simulator)
observed = estimator.run(circuit, ham_op, shots=shots).result().values[0]

print(f'QC: {observed}')
print(f'Exact: {exact}')

### 変分量子回路の設計と実装

回路とハミルトニアンが与えられた時にエネルギーを計算する手法が確立できたので、次は最低エネルギー状態を作る回路を構成することを考えます。もちろん、そんな回路が最初からわかっているわけではないので、代わりに適当なパラメータつき量子回路を設計し、その回路におけるエネルギー期待値が最も小さくなるようにパラメータの値を更新していく、つまり変分量子回路の手法を使います。

まず、近似的な低エネルギー状態を作ります。「あたり」をつけてからパラメータ付き回路部分で微調整をするようにすることで、解の探索を速くできます。

最低エネルギー状態の第0近似はいわゆるHartree-Fock状態、つまり電子が軌道を下から占有した状態です。今の場合、一番エネルギーの低い軌道は$\phi_0$と$\phi_2$（$\alpha$と$\beta$スピンの最低エネルギー軌道）なので、

$$
\psi_{\mathrm{HF}} = |\phi_0 \phi_2|.
$$

Jordan-Wigner変換で対応づけた量子レジスタで表すと

$$
\ket{\psi_{\mathrm{HF}}} = \ket{1010}
$$

となります。

In [ ]:
hf_circuit = QuantumCircuit(4)

# hf_circuitを|0000>からHartree-Fock状態|1010>を作る回路にしてください。量子ビットは右から数えることに注意。
##################
### EDIT BELOW ###
##################

hf_circuit.x([3, 1])

##################
### EDIT ABOVE ###
##################

show_state(hf_circuit, state_label=None, binary=True);

パラメータ付き回路部分の設計には最先端の理論に基づく後ろ盾と勘が必要ですが、今は手始めに業界でTwoLocalなどと呼ばれるパターンを使いましょう。

In [ ]:
from IPython.display import Image

Image(url='figs/twolocal.png', width=600)

上のような回路を1レイヤーとして、2レイヤー繰り返します。全ての$R_y$と$R_z$ゲートに独立なパラメータをあてがうので、全部で16個の自由パラメータを持つ回路になります。

回路に自由パラメータを入れる方法として、Qiskitには`Parameter`（単一のパラメータ）や`ParameterVector`（パラメータの配列）という仕組みが備わっています。下では`theta`という`ParameterVector`を定義し、`ry`や`rz`ゲートの回転角に使います。

In [ ]:
theta = ParameterVector('θ', 16)

param_circuit = QuantumCircuit(4)

# param_circuitを上で図示したTwoLocalレイヤーを2回繰り返す回路にしてください。
# thetaは通常の配列のように参照できるので、例えば第5要素をビット1のRzゲートに使うときは
#   param_circuit.ry(theta[5], 1)
# と書きます。
##################
### EDIT BELOW ###
##################

piter = iter(theta)
for ilayer in range(2):
    for iq in range(4):
        param_circuit.ry(next(piter), iq)
    
    for iq in range(4):
        param_circuit.rz(next(piter), iq)

    for ictrl in range(4):
        for itarg in range(ictrl + 1, 4):
            param_circuit.cx(ictrl, itarg)
    
##################
### EDIT ABOVE ###
##################

param_circuit.draw('mpl')

最後に基底状態の近似とパラメータ付き回路を組み合わせてアンザッツとします。

In [ ]:
ansatz = hf_circuit.compose(param_circuit, inplace=False)
ansatz.draw('mpl')

### エネルギー最小化

VQEクラスを利用してエネルギーを最小化するパラメータ値を探索します。最適化アルゴリズムにはCOBYLAを使います。また、探索の途中経過を追えるように、例によってcallback関数を定義します。

In [ ]:
def build_callback():
    estimate_history = []
    def callback(eval_count, param_values, energy, meta):
        estimate_history.append(energy)

    return callback, estimate_history

# callback関数と期待値の推移が入るリストを定義。callback中のestimatorには状態ベクトルベースのEstimatorを利用
callback, history = build_callback()

# 各パラメータを[0, π)の乱数に初期化
init = np.random.default_rng().random(ansatz.num_parameters) * np.pi

optimizer = COBYLA(maxiter=300)
vqe = VQE(estimator, ansatz, optimizer, initial_point=init, callback=callback)

In [ ]:
vqe_result = vqe.compute_minimum_eigenvalue(ham_op)
print(vqe_result)

ちなみに、今回も小さな系を扱っているので基底状態エネルギーの厳密解も計算できます。

In [ ]:
res = NumPyMinimumEigensolver().compute_minimum_eigenvalue(ham_op)
exact_ground_state_energy = res.eigenvalue

In [ ]:
def plot_loss(history):
    plt.plot(history, label='VQE estimate')
    plt.axhline(exact_ground_state_energy, color='black', label='numerical diagonalization')
    plt.xlabel('Iterations')
    plt.ylabel('Energy (Hartree)')
    plt.legend()

In [ ]:
plot_loss(history)

## Qiskit Runtime

これまでローカルのシミュレータ（AerSimulator）のみを使って計算をしてきましたが、次にクラウド上のリソースを使うことを考えてみましょう。

VQEのような量子・古典ハイブリッド計算では、量子と古典の計算リソースを交互に使うことが多くあります。この量子計算部分で、これまで講義で主に扱ってきたように回路をクラウドに投げてキューに入り、結果が帰ってくるまで待つということを繰り返していると、計算にあまりにも時間がかかってしまいます。そこで、IBM QuantumではQiskit Runtimeという仕組みが用意されています。

Qiskit Runtimeには新旧二通りの流儀があります。旧来の方法（今後フェーズアウトされる可能性あり）では、Qiskitを使ったプログラム全体を一つのジョブとしてIBM Cloud上で実行します。ユーザーはVQEなどのプログラムを書き、それを事前にPythonスクリプトとしてIBM Cloudにアップロードしておくことになります。プログラム全体が一つのジョブなので、一度実行が始まれば、プログラム中の量子回路部分もリアルタイムに（キューに入らずに即時に）結果を返します。この方法の難点としては、プログラムが常に完成されたスクリプトでなければならないこと、計算の途中経過を見づらいこと、などが挙げられます。

新しい方法では、セッションという概念が登場します。ユーザーはプログラムをローカル環境で走らせますが、量子回路を高頻度で実行しなければいけない部分に差し掛かるときに、Runtimeセッションを開始します。セッションが開かれ、最初の量子回路が（キュー待ちを経て）実行されると、その後はそのユーザーの同セッションからのジョブが最も優先的に実行されるようになります。セッションは最後の量子回路を実行してから5分、もしくは最初の回路を実行してから8時間経過した時点で閉じられます。

旧来の方法ではユーザー任意のプログラムをIBMのサーバーで実行するため、セキュリティの面などでかなり気を使うことが多いようで、おそらく今後はセッションの利用が強く推奨されていくことになると思われます。セッション利用のほうが通信などのオーバーヘッドがあり遅くなりますが、この実習でもこちらの方法を採用します。

In [ ]:
# IBMidの登録が済んでいる場合はこちら
instance = 'ibm-q-utokyo/internal/qc-training2023s'
# まだの場合はこちら
# #instance = 'ibm-q/open/main'

# いつものIBMProviderではなくQiskitRuntimeServiceを設定する
try:
    service = QiskitRuntimeService(channel='ibm_quantum', instance=instance)
except AccountNotFoundError:
    service = QiskitRuntimeService(channel='ibm_quantum', token='__paste_your_token_here__', instance=instance)

まずはクラウド上のシミュレータでセッションを試してみましょう。

In [ ]:
with Session(service=service, backend='ibmq_qasm_simulator') as session:
    qasm_estimator = RuntimeEstimator(session=session)

    callback, history = build_callback()

    # 各パラメータを[0, π)の乱数に初期化
    init = np.random.default_rng().random(ansatz.num_parameters) * np.pi

    # 試験だけなので10イテレーションで打ち切る
    optimizer = COBYLA(maxiter=10)
    vqe = VQE(qasm_estimator, ansatz, optimizer, initial_point=init, callback=callback)
    qasm_vqe_result = vqe.compute_minimum_eigenvalue(ham_op)
    print(vqe_result)

In [ ]:
plot_loss(history)

シミュレータでの実行もうまくいけば、いよいよ実機を使います。

In [ ]:
backend_list = service.backends(filters=operational_backend(min_qubits=4))
# 今一番空いているバックエンド
backend = least_busy(backend_list)
print(backend.name)

In [ ]:
with Session(service=service, backend=backend) as session:
    runtime_estimator = RuntimeEstimator(session=session)

    callback, history = build_callback()

    # 各パラメータを[0, π)の乱数に初期化
    init = np.random.default_rng().random(ansatz.num_parameters) * np.pi

    optimizer = COBYLA(maxiter=100)
    vqe = VQE(runtime_estimator, ansatz, optimizer, initial_point=init, callback=callback)
    runtime_vqe_result = vqe.compute_minimum_eigenvalue(ham_op)
    print(runtime_vqe_result)

In [ ]:
plot_loss(history)

結果はあまり厳密解に近くないかもしれません。

ただし、今回はいつものように「これだからNISQは」で片付けて諦めなくてもいいかもしれません。すでに2017年のハードウェアでまともな結果が得られているわけですから。

2017年のペーパーでは量子計算部分を極力効率化しています。おそらく最も効いているのは、分子軌道の基底を変換して実行的に二軌道の問題に直し、2量子ビット回路で問題を解いていることですが、今の4量子ビットのままでもアンザッツの工夫などでもっと良い結果が得られるはずです。是非自分で色々なアイディアを試してみてください。

ヒントとして、比較的に取り組みやすい改善法を挙げておきます。

- 今のアンザッツは電子数を保存しない（アンザッツが作る状態に"1"が2個でない計算基底も重ね合わさっている）が、水素分子のエネルギー固有ベクトルは電子2つの計算基底で張られる部分空間にあるはずである。パラメータ付き回路の構造を変えて、状態が任意のパラメータの値について$c_3 \ket{0011} + c_5 \ket{0101} + c_6 \ket{0110} + c_9 \ket{1001} + c_{10} \ket{1010} + c_{12} \ket{1100}$となるようにしてみる。
- アンザッツのCNOTの数を減らす。つまりCNOTが隣接量子ビット同士にのみかかるようにし、SWAPができるだけ起こらないようにする。
- 測定エラー緩和を導入する。
